In [1]:
using JuMP
using Gurobi
using Pkg

In [2]:
model = Model(Gurobi.Optimizer)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-09-04


A JuMP Model
├ solver: Gurobi
├ objective_sense: FEASIBILITY_SENSE
├ num_variables: 0
├ num_constraints: 0
└ Names registered in the model: none

**Fetch And Process Data**

In [3]:
using HTTP
using JSON

function fetch_data(url)
    response = HTTP.get(url)
    if response.status == 200
        return courses_data = JSON.parse(String(response.body))
    else
        println("Failed to fetch data. Status code: ", response.status)
    end
end

all_course_api = "https://fireroad.mit.edu/reference/catalog/courses/all?full=true"
courses_data = fetch_data(all_course_api)

6585-element Vector{Any}:
 Dict{String, Any}("public" => true, "out_of_class_hours" => 7.99, "rating" => 5.4, "is_variable_units" => false, "meets_with_subjects" => Any["1.001"], "pdf_option" => false, "level" => "U", "subject_id" => "1.00", "lecture_units" => 3, "offered_fall" => false…)
 Dict{String, Any}("public" => true, "out_of_class_hours" => 6.16, "rating" => 5.12, "is_variable_units" => false, "pdf_option" => false, "level" => "U", "schedule" => "Lecture,5-233/MW/0/9.30-11;Lab,5-233/F/0/9.30-11", "subject_id" => "1.000", "lecture_units" => 3, "offered_fall" => true…)
 Dict{String, Any}("public" => true, "out_of_class_hours" => 7.99, "rating" => 5.4, "is_variable_units" => false, "meets_with_subjects" => Any["1.00"], "pdf_option" => false, "level" => "G", "subject_id" => "1.001", "lecture_units" => 3, "offered_fall" => false…)
 Dict{String, Any}("public" => true, "is_variable_units" => false, "meets_with_subjects" => Any["1.147"], "pdf_option" => false, "level" => "U", "subject_

In [4]:
courses_ids = map(course -> get(course, "subject_id", nothing), courses_data)
prerequisites = map(course -> get(course, "prerequisites", nothing), courses_data)
rating = map(course -> get(course, "prerequisites", nothing), courses_data)
course_variables = Dict{String, JuMP.VariableRef}()
for course_id in courses_ids
    course_variables[course_id] = @variable(model, binary=true, base_name=course_id)
end

****Adding Prerequisite Constraints****

In [5]:
abstract type ExprNode end

struct CourseNode <: ExprNode
    course::String
end

struct ANDNode <: ExprNode
    left::ExprNode
    right::ExprNode
end

struct ORNode <: ExprNode
    left::ExprNode
    right::ExprNode
end

struct PreReq
    root::CourseNode
    prereqs::Union{ExprNode, Nothing}
end 

In [6]:
function parse_prereq_string(input)
    if input === nothing
        return
    end
    input = replace(input, r"\s+" => "")
    tokens = collect(input)
    function parse(tokens::Vector{Char})::ExprNode
        stack = ExprNode[]
        buffer = IOBuffer()
        while !isempty(tokens)
            token = popfirst!(tokens)
            if token == '('
                node = parse(tokens)
                push!(stack, node)
            elseif token == ')'
                break
            elseif token == ','
                left = isempty(stack) ? CourseNode(String(take!(buffer))) : pop!(stack)
                right = parse(tokens)
                return ANDNode(left, right)
            elseif token == '/'
                left = isempty(stack) ? CourseNode(String(take!(buffer))) : pop!(stack)
                right = parse(tokens)
                return ORNode(left, right)
            else
                write(buffer, token)
            end
        end
        return isempty(stack) ? CourseNode(String(take!(buffer))) : stack[1]
    end
    return parse(tokens)
end

parse_prereq_string (generic function with 1 method)

In [7]:
prerequisite_expressions = []
for index in 1:length(prerequisites)
    course_id = CourseNode(courses_ids[index])
    prereq_tree = parse_prereq_string(prerequisites[index])
    push!(prerequisite_expressions, PreReq(course_id, prereq_tree))
end
prerequisite_expressions

6585-element Vector{Any}:
 PreReq(CourseNode("1.00"), CourseNode("GIR:CAL1"))
 PreReq(CourseNode("1.000"), nothing)
 PreReq(CourseNode("1.001"), CourseNode("GIR:CAL1"))
 PreReq(CourseNode("1.004"), nothing)
 PreReq(CourseNode("1.005"), nothing)
 PreReq(CourseNode("1.006"), nothing)
 PreReq(CourseNode("1.007"), nothing)
 PreReq(CourseNode("1.008"), nothing)
 PreReq(CourseNode("1.009"), nothing)
 PreReq(CourseNode("1.010"), CourseNode("GIR:CAL2"))
 PreReq(CourseNode("1.010A"), CourseNode("GIR:CAL2"))
 PreReq(CourseNode("1.010B"), ORNode(CourseNode("1.010A"), CourseNode("''permissionofinstructor''")))
 PreReq(CourseNode("1.011"), nothing)
 ⋮
 PreReq(CourseNode("WGS.605"), CourseNode("''Permissionofinstructor''"))
 PreReq(CourseNode("WGS.610"), ANDNode(CourseNode("''MustapplytotheGraduateConsortiuminGender''"), ANDNode(CourseNode("''Culture''"), ANDNode(CourseNode("''Women''"), CourseNode("''Sexuality''")))))
 PreReq(CourseNode("WGS.615"), CourseNode("''MustapplytotheGraduateConsortiuminWo

In [ ]:
function add_and_or_constraints(expr::ExprNode, model::Model, course_vars::Dict{String, JuMP.VariableRef})
    if isa(expr, CourseNode)
        return get(course_vars, expr.course, 1)
    elseif isa(expr, ANDNode)
        left_var = add_and_or_constraints(expr.left, model, course_vars)
        right_var = add_and_or_constraints(expr.right, model, course_vars)
        and_var = @variable(model, binary=true)
        @constraint(model, and_var <= left_var)
        @constraint(model, and_var <= right_var)
        @constraint(model, and_var >= left_var + right_var - 1)
        return and_var
    elseif isa(expr, ORNode)
        left_var = add_and_or_constraints(expr.left, model, course_vars)
        right_var = add_and_or_constraints(expr.right, model, course_vars)
        or_var = @variable(model, binary=true)
        @constraint(model, or_var >= left_var)
        @constraint(model, or_var >= right_var)
        @constraint(model, or_var <= left_var + right_var)
        return or_var
    else
        error("Unknown expression type")
    end
end

function add_prereq_constraint(
    prereq_obj::PreReq, 
    model::Model, 
    course_vars::Dict{String, JuMP.VariableRef}
)
    course_var = get(course_vars, prereq_obj.root.course, 1)
    if prereq_obj.prereqs === nothing
        return course_var
    else
        prereq_var = add_and_or_constraints(prereq_obj.prereqs, model, course_vars)
        @constraint(model, course_var <= prereq_var)
        return course_var
    end
end

add_prereq_constraint (generic function with 1 method)

In [9]:
for expr in prerequisite_expressions
    add_prereq_constraint(expr, model, course_variables)
end